# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os as csv
import gmaps
import json
from pprint import pprint

# Import API key
from api_keys import g_key
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
path = 'output/city_weather.csv'
csvfile = pd.read_csv(path)
df = pd.DataFrame(csvfile)
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,cape town,-36.41,-71.90,50.00,81,95,9.17,CL,1600838272
1,kapaa,22.08,-159.32,80.60,69,40,13.87,US,1600838273
2,castro,-24.79,-50.01,50.88,86,75,11.21,BR,1600838066
3,ponta do sol,38.73,-28.27,64.18,85,97,23.62,PT,1600838276
4,leningradskiy,69.38,178.42,39.79,86,100,20.02,RU,1600838277
...,...,...,...,...,...,...,...,...,...
535,charters towers,38.57,-83.43,51.80,93,1,3.36,US,1600838897
536,odda,60.07,6.55,55.99,91,100,5.99,NO,1600838898
537,burnie,-41.07,145.92,55.99,57,71,15.01,AU,1600838899
538,muros,42.78,-9.06,63.00,92,84,4.00,ES,1600838900


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = df[["Lat", "Lng"]].astype(float)
weight = df["Humidity"].astype(float)

In [6]:
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, 
                                    max_intensity=100, point_radius=1)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
criteria1= df[df["Max Temp"] >= 70]
criteria2 = criteria1[criteria1["Max Temp"] <= 85]
criteria3 = criteria2[criteria2["Wind Speed"] < 10]
hotel_df = criteria3[criteria3["Cloudiness"] == 0]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,beroroha,-21.67,45.17,76.33,53,0,7.14,MG,1600838305
241,perth,-31.93,115.83,79.00,38,0,9.17,AU,1600838549
282,pochutla,15.74,-96.47,77.00,85,0,3.65,MX,1600838604
309,marmaris,36.85,28.27,75.99,73,0,3.36,TR,1600838579
337,camalu,30.85,-116.07,73.45,62,0,5.61,MX,1600838668
339,corinto,37.94,22.96,70.00,57,0,4.92,GR,1600838670
344,hami,42.80,93.45,76.50,26,0,4.36,CN,1600838676
359,goure,13.98,10.27,83.55,42,0,6.17,NE,1600838693
490,villanueva,15.32,-88.00,73.49,91,0,2.06,HN,1600838844


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df["Hotel Name"] = ''
hotel_df

C:\Users\Babyta\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
27,beroroha,-21.67,45.17,76.33,53,0,7.14,MG,1600838305,
241,perth,-31.93,115.83,79.00,38,0,9.17,AU,1600838549,
282,pochutla,15.74,-96.47,77.00,85,0,3.65,MX,1600838604,
309,marmaris,36.85,28.27,75.99,73,0,3.36,TR,1600838579,
337,camalu,30.85,-116.07,73.45,62,0,5.61,MX,1600838668,
339,corinto,37.94,22.96,70.00,57,0,4.92,GR,1600838670,
344,hami,42.80,93.45,76.50,26,0,4.36,CN,1600838676,
359,goure,13.98,10.27,83.55,42,0,6.17,NE,1600838693,
490,villanueva,15.32,-88.00,73.49,91,0,2.06,HN,1600838844,


In [11]:
params = {'radius':5000,'type': "lodging", 'key': g_key}
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    long = row['Lng']
    params['location'] = f'{lat},{long}'
    response = requests.get(url, params=params)
    hotel_name = response.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name['results'][0]['name']
    except (KeyError, IndexError,ValueError):
        print("No hotel found/Skipping...")
        hotel_df.replace("", np.nan, inplace=True)

No hotel found/Skipping...
No hotel found/Skipping...


In [12]:
hotel_df = hotel_df.dropna(subset=["Hotel Name"])
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
241,perth,-31.93,115.83,79.00,38,0,9.17,AU,1600838549,Parmelia Hilton Perth
282,pochutla,15.74,-96.47,77.00,85,0,3.65,MX,1600838604,Hotel Posada San Jose
309,marmaris,36.85,28.27,75.99,73,0,3.36,TR,1600838579,Blue Bay Platinum
337,camalu,30.85,-116.07,73.45,62,0,5.61,MX,1600838668,La Cueva del Pirata
339,corinto,37.94,22.96,70.00,57,0,4.92,GR,1600838670,Ephira Hotel
344,hami,42.80,93.45,76.50,26,0,4.36,CN,1600838676,Laoyutang Resort
490,villanueva,15.32,-88.00,73.49,91,0,2.06,HN,1600838844,Apartamentosjuarez


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))